## Brainstorming Notebook for Using LLMs to Query PostgreSQL Database

Input: Natural language prompt (e.g., "how many times did HTCP beat mathandsurf in 2022?") 
=> Use LLM to generate SQL Query based on structure of database and retrieve results from PostgreSQL database 
=> Use LLM to generate answer in natural language (e.g., "HTCP beat mathandsurf 3 times in 2022.")

Commented out the cell below, but this could be one option for how we could connect to the PostgreSQL database you have hosted on Amazon Lightsail.


In [ ]:
""" 
# Connect to the database
import psycopg2

# Database connection details
hostname = 'your_lightsail_hostname'
port = 5432
database = 'your_database_name'
username = 'your_username'
password = 'your_password'

# Connect to the database
db = psycopg2.connect(
    host=hostname,
    port=port,
    database=database,
    user=username,
    password=password
)
# Might be able to get around using psycopg2 with something like this---- Use SQLDatabase.from_uri() to connect to the database
db = SQLDatabase.from_uri(f"postgresql://{username}:{password}@{hostname}:{port}/{database}")
"""

In [70]:
# Load .env file
from dotenv import load_dotenv

load_dotenv()

True

In [54]:
from langchain.llms import OpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent

# Instantiate the OpenAI LLM model (temperature=0 means deterministic; verbose=True means print the output, note 
# we are using the LLM-version and not the chat version)
llm = OpenAI(temperature=0, verbose=True)

# Connect to the database. Using a toy dataset from https://github.com/BroccoliRaab/meleedb to play around with the idea.
db = SQLDatabase.from_uri("sqlite:///melee/static/data/meleedb.db",
                          include_tables=['melee_hitboxes', 'character_attributes'],
                          sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE character_attributes (
	"dashAndRunAccelerationA" FLOAT, 
	"bubbleRatio" INTEGER, 
	friction FLOAT, 
	"airMobilityB" FLOAT, 
	"runAcceleration" INTEGER, 
	"kirbyStarDamage" INTEGER, 
	"numberOfJumps" INTEGER, 
	"jumpHInitialVelocity" FLOAT, 
	"walkAcceleration" FLOAT, 
	"jab3Window?" INTEGER, 
	"doubleJumpMomentum" FLOAT, 
	"initialWalkVelocity" FLOAT, 
	"jumpVInitialVelocity" FLOAT, 
	gravity FLOAT, 
	"walkMaximumVelocity" FLOAT, 
	"maxAerialHVelocity" FLOAT, 
	"iceTraction?" INTEGER, 
	"walljumpVVelocity" FLOAT, 
	"runAnimationScaling" FLOAT, 
	"walljumpHVelocity" FLOAT, 
	"jumpStartup" INTEGER, 
	character VARCHAR, 
	"rapidJabWindow" INTEGER, 
	"victoryScreenWindowModelScaling" INTEGER, 
	"dairLandingLag" INTEGER, 
	weight INTEGER, 
	"bairLandingLag" INTEGER, 
	"ledgejumpHorizontalVelocity" INTEGER, 
	"fairLandingLag" INTEGER, 
	"shorthopVInitialVelocity" FLOAT, 
	"cameraTargetZoomBone" INTEGER, 
	"airJumpMultiplier" FLOAT, 
	"normalLandingLag" INTEGER, 
	"specialJumpA

In [57]:
from langchain.prompts.prompt import PromptTemplate
from langchain import SQLDatabaseChain
from langchain.callbacks import get_openai_callback

DEFAULT_TEMPLATE = """Given an input question {input}, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}
"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True, use_query_checker=True)



with get_openai_callback() as cb:
    result = db_chain.run("What is the average weight of all characters?")
    print(cb)



> Entering new  chain...
What is the average weight of all characters?
SQLQuery:SELECT AVG(weight) FROM character_attributes;
SQLResult: [(89.92592592592592,)]
Answer:Question: What is the average weight of all characters?
SQLQuery: SELECT AVG(weight) FROM character_attributes;
> Finished chain.
Tokens Used: 6600
	Prompt Tokens: 6533
	Completion Tokens: 67
Successful Requests: 3
Total Cost (USD): $0.132


That last call was pretty expensive, $0.13 for one query because it sent the whole database in.
Wonder if it'd be possible to use an open-source model and fine-tune it on the db.

Below I'll run a query that will be VERY slow on the first run. Likely when switching this over to production would want to put the loading of the model
into one function and query calls in another. Larger the model, much longer it takes to load. I've heard flan-t5-xl is somewhat decent to work with.

In [77]:
## Still expensive.. let's try with an open source model from HuggingFace Hub
# I'm getting time-out errors, will try again with a different model.

import os
HUGGINGFACEHUB_API_TOKEN = os.environ.get('HUGGINGFACEHUB_API_TOKEN')

from langchain import HuggingFaceHub

repo_id = "google/flan-t5-xl" 

llm_flan = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0.1}, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)

db_chain_flan = SQLDatabaseChain.from_llm(llm_flan, db, prompt=PROMPT, verbose=True, use_query_checker=True)
with get_openai_callback() as cb:
    result = db_chain_flan.run("What is the average weight of all characters?")
    print(cb)





> Entering new  chain...
What is the average weight of all characters?
SQLQuery:

ValueError: Error raised by inference API: Model google/flan-t5-xl time out